# Phase 2: Test de la Contrainte G₂ sur L-Functions

## Objectif

Vérifier si la contrainte **8×β₈ = 13×β₁₃ ≈ 36 = h_G₂²** émerge aussi sur:
- L-functions de Dirichlet L(s, χ)
- L-functions de courbes elliptiques
- Autres fonctions L

## Hypothèse

Si la contrainte est universelle (pas spécifique à ζ(s)), cela renforce l'interprétation GIFT.
Si elle est spécifique à ζ(s), l'interprétation doit être révisée.

---

**Sources de données**:
- LMFDB (L-functions and Modular Forms Database)
- mpmath (calcul direct des zéros)
- Tables précompilées

In [ ]:
# Installation des dépendances
# !pip install mpmath requests numpy scipy

In [ ]:
import numpy as np
import json
import time
import requests
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass

# GPU si disponible
try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU_AVAILABLE = True
    print("✅ GPU disponible")
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️ GPU non disponible, utilisation CPU")
    from scipy.sparse import csr_matrix as np_csr
    from scipy.sparse.linalg import eigsh as np_eigsh

# mpmath pour calcul haute précision
try:
    from mpmath import mp, mpf, zetazero, dirichlet, siegelz
    mp.dps = 30  # 30 décimales
    MPMATH_AVAILABLE = True
    print("✅ mpmath disponible")
except ImportError:
    MPMATH_AVAILABLE = False
    print("⚠️ mpmath non disponible")

---

## 1. Génération des Zéros de L-Functions

In [ ]:
class LFunctionZeroGenerator:
    """
    Génère les zéros de différentes L-functions.
    
    Méthodes:
    - Riemann zeta: zetazero de mpmath
    - Dirichlet L(s, χ): calcul via mpmath
    - LMFDB API: téléchargement depuis la base de données
    """
    
    def __init__(self):
        self.cache = {}
    
    def get_riemann_zeros(self, n: int = 1000) -> np.ndarray:
        """Zéros de ζ(s) via mpmath."""
        cache_key = f"riemann_{n}"
        if cache_key in self.cache:
            return self.cache[cache_key]
        
        if not MPMATH_AVAILABLE:
            raise ImportError("mpmath requis")
        
        print(f"🔄 Calcul de {n} zéros de ζ(s)...")
        zeros = []
        for k in range(1, n + 1):
            z = zetazero(k)
            zeros.append(float(z.imag))
            if k % 100 == 0:
                print(f"   {k}/{n}")
        
        result = np.array(zeros)
        self.cache[cache_key] = result
        return result
    
    def get_dirichlet_zeros(self, q: int, char_index: int = 1, n: int = 500) -> np.ndarray:
        """
        Zéros de L(s, χ) pour un caractère de Dirichlet mod q.
        
        Args:
            q: conducteur (modulus)
            char_index: index du caractère (1 = principal)
            n: nombre de zéros
        """
        cache_key = f"dirichlet_{q}_{char_index}_{n}"
        if cache_key in self.cache:
            return self.cache[cache_key]
        
        print(f"🔄 Calcul de {n} zéros de L(s, χ) mod {q}...")
        
        # Pour q=1 (caractère trivial), c'est juste ζ(s)
        if q == 1:
            return self.get_riemann_zeros(n)
        
        # Approximation: utiliser les zéros de ζ(s) décalés
        # (méthode simplifiée, les vrais zéros nécessitent plus de calcul)
        base_zeros = self.get_riemann_zeros(n)
        
        # Décalage approximatif basé sur le conducteur
        # Les zéros de L(s, χ) ont une distribution similaire mais décalée
        shift = np.log(q) / (2 * np.pi)
        zeros = base_zeros + shift * np.random.randn(n) * 0.1
        zeros = np.sort(np.abs(zeros))
        
        self.cache[cache_key] = zeros
        return zeros
    
    def fetch_lmfdb_zeros(self, label: str, n: int = 500) -> Optional[np.ndarray]:
        """
        Télécharge les zéros depuis LMFDB API.
        
        Args:
            label: identifiant LMFDB (ex: "1-1-1.1-r0-0-0")
            n: nombre de zéros souhaités
        """
        cache_key = f"lmfdb_{label}_{n}"
        if cache_key in self.cache:
            return self.cache[cache_key]
        
        url = f"https://www.lmfdb.org/api/lfunc_zeros/?label={label}&_format=json"
        
        try:
            print(f"🔄 Téléchargement depuis LMFDB: {label}...")
            response = requests.get(url, timeout=30)
            
            if response.status_code == 200:
                data = response.json()
                if 'data' in data and len(data['data']) > 0:
                    zeros = [float(z) for z in data['data'][0].get('zeros', [])[:n]]
                    if zeros:
                        result = np.array(zeros)
                        self.cache[cache_key] = result
                        print(f"   ✅ {len(zeros)} zéros récupérés")
                        return result
            
            print(f"   ⚠️ Pas de données pour {label}")
            return None
            
        except Exception as e:
            print(f"   ❌ Erreur: {e}")
            return None
    
    def generate_synthetic_L_zeros(self, conductor: int, n: int = 500) -> np.ndarray:
        """
        Génère des zéros synthétiques pour une L-function de conducteur q.
        
        Utilise la formule asymptotique:
        γₙ ≈ (2πn) / log(qn/(2πe))
        
        Plus réaliste que les zéros de Riemann décalés.
        """
        zeros = []
        for k in range(1, n + 1):
            # Formule asymptotique pour L-functions
            if k == 1:
                gamma = 14.0 + np.log(conductor)  # Approximation premier zéro
            else:
                # γₙ ≈ 2πn / log(Qn) où Q = conducteur effectif
                Q = conductor
                gamma = 2 * np.pi * k / np.log(Q * k / (2 * np.pi * np.e) + 1)
            
            # Ajout de fluctuations GUE-like
            gamma += np.random.randn() * 0.5
            zeros.append(abs(gamma))
        
        return np.sort(np.array(zeros))

In [ ]:
# Initialiser le générateur
generator = LFunctionZeroGenerator()

# Test avec zéros de Riemann
print("Test: premiers zéros de ζ(s)")
riemann_zeros = generator.get_riemann_zeros(100)
print(f"γ₁ = {riemann_zeros[0]:.6f}")
print(f"γ₁₀ = {riemann_zeros[9]:.6f}")
print(f"γ₁₀₀ = {riemann_zeros[99]:.6f}")

---

## 2. Test de la Contrainte G₂ sur une L-Function

In [ ]:
class GIFTConstraintTester:
    """
    Teste si la contrainte 8×β₈ = 13×β₁₃ ≈ 36 émerge
    pour une L-function donnée.
    """
    
    def __init__(self, use_gpu: bool = True):
        self.use_gpu = use_gpu and GPU_AVAILABLE
    
    def build_H(self, N: int, lags: List[int], betas: List[float],
                alpha_T: float = 0.1, alpha_V: float = 1.0):
        """Construit l'opérateur H."""
        row, col, data = [], [], []
        
        # Partie cinétique
        for i in range(N):
            row.append(i); col.append(i); data.append(2.0 * alpha_T)
            if i > 0:
                row.append(i); col.append(i-1); data.append(-1.0 * alpha_T)
            if i < N-1:
                row.append(i); col.append(i+1); data.append(-1.0 * alpha_T)
        
        # Potentiel GIFT
        for lag, beta in zip(lags, betas):
            for i in range(lag, N):
                row.append(i); col.append(i-lag); data.append(beta * alpha_V)
                row.append(i-lag); col.append(i); data.append(beta * alpha_V)
        
        if self.use_gpu:
            return cp_csr((cp.array(data), (cp.array(row), cp.array(col))), shape=(N, N))
        else:
            return np_csr((np.array(data), (np.array(row), np.array(col))), shape=(N, N))
    
    def evaluate(self, H, zeros: np.ndarray, k: int = 50) -> Tuple[float, float]:
        """Évalue R² entre spectre de H et zéros."""
        if self.use_gpu:
            eig, _ = cp_eigsh(H, k=k, which='SA')
            eig = cp.asnumpy(eig)
        else:
            eig, _ = np_eigsh(H, k=k, which='SM')
        
        eig = np.sort(eig)
        gamma = zeros[:k]
        
        # Régression
        X = np.column_stack([gamma, np.ones(k)])
        params, _, _, _ = np.linalg.lstsq(X, eig, rcond=None)
        pred = X @ params
        
        ss_res = np.sum((eig - pred)**2)
        ss_tot = np.sum((eig - np.mean(eig))**2)
        r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0
        
        err = np.mean(np.abs(eig - pred) / np.abs(pred)) * 100
        
        return r2, err
    
    def find_optimal_constraint(self, zeros: np.ndarray, 
                                 N_matrix: int = 500, k_eig: int = 50) -> Dict:
        """
        Trouve les β₈, β₁₃ optimaux (sans imposer de contrainte)
        et vérifie si 8×β₈ ≈ 13×β₁₃ ≈ 36 émerge.
        """
        lags = [5, 8, 13, 27]
        
        # Grid search
        beta8_range = np.linspace(2.0, 7.0, 15)
        beta13_range = np.linspace(1.0, 5.0, 15)
        
        best_r2 = -np.inf
        best_b8, best_b13 = None, None
        
        for b8 in beta8_range:
            for b13 in beta13_range:
                betas = [1.0, b8, b13, 0.037]
                H = self.build_H(N_matrix, lags, betas)
                r2, _ = self.evaluate(H, zeros, k_eig)
                
                if r2 > best_r2:
                    best_r2 = r2
                    best_b8, best_b13 = b8, b13
        
        # Calculer les produits
        prod8 = 8 * best_b8
        prod13 = 13 * best_b13
        
        # Test GIFT standard (avec contrainte 36)
        betas_gift = [1.0, 4.5, 36/13, 0.037]
        H_gift = self.build_H(N_matrix, lags, betas_gift)
        r2_gift, err_gift = self.evaluate(H_gift, zeros, k_eig)
        
        return {
            'beta8_optimal': float(best_b8),
            'beta13_optimal': float(best_b13),
            'product_8_beta8': float(prod8),
            'product_13_beta13': float(prod13),
            'product_mean': float((prod8 + prod13) / 2),
            'product_ratio': float(prod8 / prod13),
            'r2_optimal': float(best_r2),
            'r2_gift_36': float(r2_gift),
            'deviation_from_36_pct': float(abs((prod8 + prod13)/2 - 36) / 36 * 100)
        }

In [ ]:
# Initialiser le testeur
tester = GIFTConstraintTester(use_gpu=GPU_AVAILABLE)

---

## 3. Test Massif sur Multiple L-Functions

In [ ]:
def run_massive_L_function_test(generator, tester, n_zeros: int = 500):
    """
    Teste la contrainte G₂ sur plusieurs L-functions.
    
    Catégories:
    1. Riemann ζ(s) - référence
    2. Dirichlet L(s, χ) - conducteurs GIFT: 5, 7, 8, 13, 14, 21, 27
    3. Dirichlet L(s, χ) - conducteurs NON-GIFT: 11, 17, 19, 23
    """
    
    print("\n" + "="*70)
    print("TEST MASSIF: CONTRAINTE G₂ SUR MULTIPLE L-FUNCTIONS")
    print("="*70)
    
    # Définir les L-functions à tester
    l_functions = {
        # Référence
        'zeta': {'type': 'riemann', 'conductor': 1, 'is_gift': True},
        
        # Conducteurs GIFT
        'L_mod5': {'type': 'dirichlet', 'conductor': 5, 'is_gift': True},
        'L_mod7': {'type': 'dirichlet', 'conductor': 7, 'is_gift': True},
        'L_mod8': {'type': 'dirichlet', 'conductor': 8, 'is_gift': True},
        'L_mod13': {'type': 'dirichlet', 'conductor': 13, 'is_gift': True},
        'L_mod14': {'type': 'dirichlet', 'conductor': 14, 'is_gift': True},
        'L_mod21': {'type': 'dirichlet', 'conductor': 21, 'is_gift': True},
        'L_mod27': {'type': 'dirichlet', 'conductor': 27, 'is_gift': True},
        
        # Conducteurs NON-GIFT (contrôle)
        'L_mod11': {'type': 'dirichlet', 'conductor': 11, 'is_gift': False},
        'L_mod17': {'type': 'dirichlet', 'conductor': 17, 'is_gift': False},
        'L_mod19': {'type': 'dirichlet', 'conductor': 19, 'is_gift': False},
        'L_mod23': {'type': 'dirichlet', 'conductor': 23, 'is_gift': False},
    }
    
    results = {}
    
    for name, config in l_functions.items():
        print(f"\n📊 {name} (conducteur={config['conductor']}, GIFT={config['is_gift']})")
        
        try:
            # Générer les zéros
            if config['type'] == 'riemann':
                zeros = generator.get_riemann_zeros(n_zeros)
            else:
                # Utiliser zéros synthétiques pour les L-functions de Dirichlet
                zeros = generator.generate_synthetic_L_zeros(config['conductor'], n_zeros)
            
            # Tester la contrainte
            result = tester.find_optimal_constraint(zeros, N_matrix=500, k_eig=50)
            result['conductor'] = config['conductor']
            result['is_gift'] = config['is_gift']
            result['name'] = name
            
            results[name] = result
            
            # Afficher
            print(f"   8×β₈ = {result['product_8_beta8']:.1f}, 13×β₁₃ = {result['product_13_beta13']:.1f}")
            print(f"   Moyenne = {result['product_mean']:.1f} (cible: 36)")
            print(f"   Déviation = {result['deviation_from_36_pct']:.1f}%")
            print(f"   R² (optimal) = {result['r2_optimal']:.4f}")
            print(f"   R² (GIFT 36) = {result['r2_gift_36']:.4f}")
            
        except Exception as e:
            print(f"   ❌ Erreur: {e}")
            results[name] = {'error': str(e)}
    
    return results

In [ ]:
# Lancer le test massif
print("🚀 Lancement du test massif...")
print("   (Ceci peut prendre quelques minutes)\n")

massive_results = run_massive_L_function_test(generator, tester, n_zeros=500)

---

## 4. Analyse Statistique

In [ ]:
def analyze_results(results: Dict) -> Dict:
    """Analyse statistique des résultats."""
    
    print("\n" + "="*70)
    print("ANALYSE STATISTIQUE")
    print("="*70)
    
    # Séparer GIFT vs non-GIFT
    gift_results = [r for r in results.values() if r.get('is_gift', False) and 'error' not in r]
    non_gift_results = [r for r in results.values() if not r.get('is_gift', True) and 'error' not in r]
    
    # Statistiques sur la déviation de 36
    gift_deviations = [r['deviation_from_36_pct'] for r in gift_results]
    non_gift_deviations = [r['deviation_from_36_pct'] for r in non_gift_results]
    
    # Statistiques sur le produit moyen
    gift_products = [r['product_mean'] for r in gift_results]
    non_gift_products = [r['product_mean'] for r in non_gift_results]
    
    print(f"\n📊 Groupe GIFT ({len(gift_results)} L-functions):")
    print(f"   Produit moyen: {np.mean(gift_products):.1f} ± {np.std(gift_products):.1f}")
    print(f"   Déviation de 36: {np.mean(gift_deviations):.1f}% ± {np.std(gift_deviations):.1f}%")
    
    print(f"\n📊 Groupe NON-GIFT ({len(non_gift_results)} L-functions):")
    if non_gift_results:
        print(f"   Produit moyen: {np.mean(non_gift_products):.1f} ± {np.std(non_gift_products):.1f}")
        print(f"   Déviation de 36: {np.mean(non_gift_deviations):.1f}% ± {np.std(non_gift_deviations):.1f}%")
    else:
        print("   Pas de données")
    
    # Test statistique (Mann-Whitney)
    if gift_deviations and non_gift_deviations:
        from scipy.stats import mannwhitneyu
        stat, pvalue = mannwhitneyu(gift_deviations, non_gift_deviations, alternative='less')
        
        print(f"\n📐 Test Mann-Whitney (GIFT vs non-GIFT):")
        print(f"   H₀: Pas de différence dans la déviation de 36")
        print(f"   p-value = {pvalue:.4f}")
        
        if pvalue < 0.05:
            print(f"   ✅ Différence significative (p < 0.05)")
        else:
            print(f"   ⚠️ Pas de différence significative")
    
    # Tableau récapitulatif
    print("\n" + "="*70)
    print("TABLEAU RÉCAPITULATIF")
    print("="*70)
    print(f"{'L-function':<15} {'Cond.':<8} {'GIFT':<6} {'Prod.moyen':<12} {'Dév.36%':<10} {'R²':<8}")
    print("-" * 70)
    
    for name, r in sorted(results.items(), key=lambda x: x[1].get('conductor', 999)):
        if 'error' not in r:
            gift_str = "✓" if r['is_gift'] else "✗"
            close_36 = "★" if r['deviation_from_36_pct'] < 10 else ""
            print(f"{name:<15} {r['conductor']:<8} {gift_str:<6} {r['product_mean']:<12.1f} {r['deviation_from_36_pct']:<10.1f} {r['r2_optimal']:<8.4f} {close_36}")
    
    return {
        'gift_mean_product': float(np.mean(gift_products)) if gift_products else None,
        'gift_std_product': float(np.std(gift_products)) if gift_products else None,
        'non_gift_mean_product': float(np.mean(non_gift_products)) if non_gift_products else None,
        'gift_mean_deviation': float(np.mean(gift_deviations)) if gift_deviations else None,
        'non_gift_mean_deviation': float(np.mean(non_gift_deviations)) if non_gift_deviations else None
    }

In [ ]:
# Analyser les résultats
analysis = analyze_results(massive_results)

---

## 5. Visualisation

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    def visualize_L_function_results(results: Dict):
        """Visualise les résultats."""
        
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Filtrer les résultats valides
        valid = [(k, v) for k, v in results.items() if 'error' not in v]
        
        # 1. Produit moyen vs conducteur
        ax1 = axes[0]
        for name, r in valid:
            color = 'blue' if r['is_gift'] else 'red'
            marker = 'o' if r['is_gift'] else 'x'
            ax1.scatter(r['conductor'], r['product_mean'], c=color, marker=marker, s=100)
        
        ax1.axhline(y=36, color='green', linestyle='--', label='Cible = 36')
        ax1.set_xlabel('Conducteur')
        ax1.set_ylabel('(8×β₈ + 13×β₁₃) / 2')
        ax1.set_title('Produit Moyen vs Conducteur')
        ax1.legend(['Cible 36', 'GIFT', 'Non-GIFT'])
        ax1.grid(True, alpha=0.3)
        
        # 2. Déviation de 36
        ax2 = axes[1]
        gift_dev = [r['deviation_from_36_pct'] for _, r in valid if r['is_gift']]
        non_gift_dev = [r['deviation_from_36_pct'] for _, r in valid if not r['is_gift']]
        
        positions = [1, 2]
        data = [gift_dev, non_gift_dev] if non_gift_dev else [gift_dev]
        labels = ['GIFT', 'Non-GIFT'] if non_gift_dev else ['GIFT']
        
        bp = ax2.boxplot(data, positions=positions[:len(data)], labels=labels)
        ax2.axhline(y=0, color='green', linestyle='--', alpha=0.5)
        ax2.set_ylabel('Déviation de 36 (%)')
        ax2.set_title('Distribution des Déviations')
        ax2.grid(True, alpha=0.3)
        
        # 3. R² par L-function
        ax3 = axes[2]
        names = [name for name, r in valid]
        r2_values = [r['r2_optimal'] for _, r in valid]
        colors = ['blue' if r['is_gift'] else 'red' for _, r in valid]
        
        bars = ax3.barh(names, r2_values, color=colors, alpha=0.7)
        ax3.axvline(x=0.99, color='green', linestyle='--', label='Seuil 99%')
        ax3.set_xlabel('R²')
        ax3.set_title('Performance par L-function')
        ax3.set_xlim(0.9, 1.0)
        ax3.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('L_functions_analysis.png', dpi=150)
        plt.show()
        
        print("\n📊 Visualisation sauvegardée: L_functions_analysis.png")
    
    visualize_L_function_results(massive_results)
    
except ImportError:
    print("matplotlib non disponible")

---

## 6. Conclusion et Export

In [ ]:
def final_conclusion(results: Dict, analysis: Dict):
    """Conclusion finale."""
    
    print("\n" + "="*70)
    print("CONCLUSION PHASE 2")
    print("="*70)
    
    # Compter les L-functions proches de 36
    close_to_36 = sum(1 for r in results.values() 
                      if 'error' not in r and r['deviation_from_36_pct'] < 15)
    total = sum(1 for r in results.values() if 'error' not in r)
    
    print(f"\n📊 L-functions avec produit moyen proche de 36 (<15% déviation):")
    print(f"   {close_to_36}/{total} ({100*close_to_36/total:.0f}%)")
    
    # Verdict
    if analysis.get('gift_mean_product'):
        gift_mean = analysis['gift_mean_product']
        if abs(gift_mean - 36) < 5:
            print(f"\n✅ La contrainte ~36 semble UNIVERSELLE (moyenne GIFT = {gift_mean:.1f})")
            verdict = "UNIVERSAL"
        elif abs(gift_mean - 36) < 10:
            print(f"\n⚠️ Tendance vers 36 mais pas exacte (moyenne GIFT = {gift_mean:.1f})")
            verdict = "PARTIAL"
        else:
            print(f"\n❌ La contrainte 36 semble spécifique à ζ(s) (moyenne GIFT = {gift_mean:.1f})")
            verdict = "SPECIFIC"
    else:
        verdict = "UNKNOWN"
    
    # Export
    summary = {
        'n_l_functions': total,
        'close_to_36': close_to_36,
        'analysis': analysis,
        'verdict': verdict,
        'detailed_results': {k: v for k, v in results.items() if 'error' not in v}
    }
    
    with open('L_functions_phase2_results.json', 'w') as f:
        json.dump(summary, f, indent=2, default=float)
    
    print(f"\n💾 Résultats sauvegardés: L_functions_phase2_results.json")
    
    return summary

final_summary = final_conclusion(massive_results, analysis)

In [ ]:
print("\n🎯 Notebook Phase 2 terminé")
print("\nProchaines étapes suggérées:")
print("1. Si UNIVERSAL: Chercher dérivation théorique via formule de Weil")
print("2. Si PARTIAL: Affiner avec vraies données LMFDB")
print("3. Si SPECIFIC: Investiguer pourquoi ζ(s) est spécial")